In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, SessionNotCreatedException, InvalidSessionIdException
import time
import pandas as pd


def dismiss_consent_popup(driver):
    try:
        consent_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.fc-cta-consent"))
        )
        consent_btn.click()
        print("✅ Consent popup dismissed.")
    except TimeoutException:
        print("⏩ No consent popup found.")


def navigate_to_first_gameweek(driver, max_clicks=40):
    """Navigate to the first gameweek by clicking 'Load previous' until no more available"""
    print("🚀 Navigating to first gameweek...")
    
    for click_num in range(max_clicks):
        try:
            time.sleep(2)  # Shorter wait since we're not extracting data
            
            load_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'Load previous')]"))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_btn)
            time.sleep(1)
            load_btn.click()
            print(f"🖱️ Click {click_num + 1}: Loaded previous gameweek")
            
        except TimeoutException:
            print(f"✅ Reached the beginning! Total clicks: {click_num}")
            break
        except Exception as e:
            print(f"❌ Error during navigation: {e}")
            break
    
    print("🎯 Now at the first gameweek. Ready to extract all data!")


def extract_all_data_from_current_page(driver):
    """Extract all match data from the current page (which should now contain all gameweeks)"""
    print("📊 Extracting all match data...")
    
    all_matches = []
    containers = driver.find_elements(By.CSS_SELECTOR, "div[data-known-size]")
    current_date = ""
    
    try:
        game_week_elem = driver.find_element(By.CSS_SELECTOR, "div.sc-3cef03db-0 span.label")
        game_week_text = game_week_elem.text.strip()
    except:
        game_week_text = ""

    for container in containers:
        try:
            # Skip if no match link exists
            if not container.find_elements(By.CSS_SELECTOR, "a.sc-22ef6ec-0"):
                continue
                
            # Get date if present
            date_el = container.find_elements(By.CSS_SELECTOR, "span.label")
            if date_el and "2025" in date_el[0].text:
                current_date = date_el[0].text.strip()
                continue

            # Extract basic match data
            link = container.find_element(By.CSS_SELECTOR, "a.sc-22ef6ec-0").get_attribute("href")
            teams = [span.text.strip() for span in container.find_elements(By.CSS_SELECTOR, "span.label.sc-8bec8ea9-3.RA-dqA") 
                    if span.text.strip() and span.text.strip() not in ['FT', 'HT', 'LIVE']]
            scores = [s.text.strip() for s in container.find_elements(By.CSS_SELECTOR, "span.label.score") 
                     if s.text.strip().isdigit()]

            # Only process if we have complete data
            if len(teams) >= 2 and len(scores) >= 2:
                home_score, away_score = int(scores[0]), int(scores[1])
                if home_score + away_score <= 3:  # Filter for ≤3 goals
                    all_matches.append({
                        "Date": current_date,
                        "Home": teams[0],
                        "Away": teams[1], 
                        "Home_Score": home_score,
                        "Away_Score": away_score,
                        "Final_Score": f"{home_score}-{away_score}",
                        "Link": link if link.startswith("http") else f"https://ca.soccerway.com{link}"
                    })
        except:
            continue  # Skip any problematic containers

    print(f"✅ Extracted {len(all_matches)} matches (≤3 total goals).")
    return all_matches


# ---------- MAIN EXECUTION ----------

url = "https://ca.soccerway.com/national/france/ligue-1/20242025/regular-season/r81802/matches/"

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

try:
    print("🌐 Loading initial page...")
    driver.get(url)
    dismiss_consent_popup(driver)

    # Phase 1: Navigate to the beginning (fast)
    nav_start = time.time()
    navigate_to_first_gameweek(driver, max_clicks=40)
    nav_time = time.time() - nav_start
    print(f"⏱️ Navigation took: {nav_time:.2f} seconds")
    
    # Phase 2: Extract all data in one go (efficient)
    extract_start = time.time()
    all_matches = extract_all_data_from_current_page(driver)
    extract_time = time.time() - extract_start
    print(f"⏱️ Data extraction took: {extract_time:.2f} seconds")

    # ---------- CREATE DATAFRAME ----------
    ligue1_data_frame = pd.DataFrame(all_matches)
    pd.set_option('display.max_colwidth', None)

    print("📊 DataFrame created!")
    print(ligue1_data_frame.head())

    # ---------- ADD FIRST GOAL MINUTES ----------
    def get_first_goal_minute(match_url):
        print("Extracting minutes of first goal score, this could take a while!")
        try:
            driver.get(match_url)
            time.sleep(2)
            goal_times = driver.find_elements(By.CSS_SELECTOR, "span.label.info-inverted")
            if goal_times:
                first_goal = goal_times[0].text.strip().replace("'", "")
                
                if '+' in first_goal:
                    # Split "90+3" into ["90", "3"] and add them
                    parts = first_goal.split('+')
                    if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                        return int(parts[0]) + int(parts[1])
                else:
                    # Regular minute like "18"
                    return int(first_goal) if first_goal.isdigit() else None
            return None
        except:
            return None

    ligue1_data_frame['first_goal_min'] = None
    
    goal_start = time.time()
    matches_with_goals = ligue1_data_frame[ligue1_data_frame['Home_Score'] + ligue1_data_frame['Away_Score'] > 0]
    
    for idx, row in matches_with_goals.iterrows():
        first_goal_min = get_first_goal_minute(row['Link'])
        ligue1_data_frame.at[idx, 'first_goal_min'] = first_goal_min
    
    goal_time = time.time() - goal_start
    print(f"⏱️ First goal extraction took: {goal_time:.2f} seconds ({len(matches_with_goals)} matches)")

    # ---------- SAVE CSV ----------
    ligue1_data_frame.to_csv("ligue1_2024_matches.csv", index=False)
    print("💾 Data saved to ligue1_2024_matches.csv")
    
    # Total time summary
    total_time = nav_time + extract_time + goal_time
    print(f"\n📊 TOTAL TIME: {total_time:.2f} seconds")
    print(f"   - Navigation: {nav_time:.2f}s")
    print(f"   - Extraction: {extract_time:.2f}s") 
    print(f"   - Goal minutes: {goal_time:.2f}s")

except Exception as e:
    print(f"🚨 Script failed: {e}")
finally:
    driver.quit()
    print("🚪 Browser closed")

🌐 Loading initial page...
✅ Consent popup dismissed.
🚀 Navigating to first gameweek...
🖱️ Click 1: Loaded previous gameweek
🖱️ Click 2: Loaded previous gameweek
🖱️ Click 3: Loaded previous gameweek
🖱️ Click 4: Loaded previous gameweek
🖱️ Click 5: Loaded previous gameweek
🖱️ Click 6: Loaded previous gameweek
🖱️ Click 7: Loaded previous gameweek
🖱️ Click 8: Loaded previous gameweek
🖱️ Click 9: Loaded previous gameweek
🖱️ Click 10: Loaded previous gameweek
🖱️ Click 11: Loaded previous gameweek
🖱️ Click 12: Loaded previous gameweek
🖱️ Click 13: Loaded previous gameweek
🖱️ Click 14: Loaded previous gameweek
🖱️ Click 15: Loaded previous gameweek
🖱️ Click 16: Loaded previous gameweek
🖱️ Click 17: Loaded previous gameweek
🖱️ Click 18: Loaded previous gameweek
🖱️ Click 19: Loaded previous gameweek
🖱️ Click 20: Loaded previous gameweek
🖱️ Click 21: Loaded previous gameweek
🖱️ Click 22: Loaded previous gameweek
🖱️ Click 23: Loaded previous gameweek
🖱️ Click 24: Loaded previous gameweek
🖱️ Click 2

In [12]:
finalDataFrame = ligue1_data_frame[ligue1_data_frame['first_goal_min'] >= 70]
finalDataFrame.count()

Date              28
Home              28
Away              28
Home_Score        28
Away_Score        28
Final_Score       28
Link              28
first_goal_min    28
dtype: int64